# Pacotes Python

In [3]:
import pandas  as pd
import numpy   as np
import seaborn as sns
from google.colab import drive
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Leitura Data Sets

In [5]:
link1 = '/content/gdrive/MyDrive/Colab Notebooks/Day7/DNC_order_items_dataset.csv'
df1 = pd.read_csv(link1,index_col='Unnamed: 0')

In [78]:
df1.head()

,order_id,order_item_id,product_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90


In [7]:
link2 = '/content/gdrive/MyDrive/Colab Notebooks/Day7/DNC_order_reviews_dataset.csv'
df2 = pd.read_csv(link2,index_col='Unnamed: 0')

In [35]:
df2.head()

,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


In [9]:
link3 = '/content/gdrive/MyDrive/Colab Notebooks/Day7/DNC_orders_dataset.csv'
df3 = pd.read_csv(link3,index_col='Unnamed: 0')

In [37]:
df3.head()

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered


In [11]:
link4 = '/content/gdrive/MyDrive/Colab Notebooks/Day7/DNC_products_dataset.csv'
df4 = pd.read_csv(link4,index_col='Unnamed: 0')

In [73]:
df4.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,625.0,20.0,17.0,13.0


In [36]:
df0 = df1.merge(df2, on=['order_id'])
df0.shape

(112372, 6)

# Concatenando Data Sets

In [99]:
df_concat = df1.merge(df2, on=['order_id'], how='inner')
df_concat = df_concat.merge(df3, on=['order_id'], how='inner')
df_concat = df_concat.merge(df4, on=['product_id'], how='inner')
df_concat.head()

,order_id,order_item_id,product_id,price,review_id,review_score,customer_id,order_status,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.9,97ca439bc427b48bc1cd7177abe71365,5,3ce436f183e68e07877b285a838db11a,delivered,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,55.9,b11cba360bbe71410c291b764753d37f,5,e6eecc5a77de221464d1c4eaff0a9b64,delivered,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,64.9,af01c4017c5ab46df6cc810e069e654a,4,4ef55bf80f711b372afebcb7c715344a,delivered,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,58.9,8304ff37d8b16b57086fa283fe0c44f8,5,30407a72ad8b3f4df4d15369126b20c9,delivered,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,58.9,426f43a82185969503fb3c86241a9535,5,91a792fef70ecd8cc69d3c7feb3d12da,delivered,cool_stuff,58.0,598.0,650.0,28.0,9.0,14.0


# Análise e Tratamento do Data Set Concatenado

In [100]:
df_concat.shape

(112372, 15)

In [101]:
df_concat.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
price                         float64
review_id                      object
review_score                    int64
customer_id                    object
order_status                   object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [102]:
df_concat['order_id'] = df_concat['order_id'].astype('string')
df_concat['customer_id'] = df_concat['customer_id'].astype('string')
df_concat['order_status'] = df_concat['order_status'].astype('string')
df_concat['product_category_name'] = df_concat['product_category_name'].astype('string')
df_concat['product_id'] = df_concat['product_id'].astype('string')
df_concat['review_id'] = df_concat['review_id'].astype('string')

In [103]:
df_concat.dtypes

order_id                       string
order_item_id                   int64
product_id                     string
price                         float64
review_id                      string
review_score                    int64
customer_id                    string
order_status                   string
product_category_name          string
product_name_lenght           float64
product_description_lenght    float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [104]:
df_concat.isna().sum()

order_id                         0
order_item_id                    0
product_id                       0
price                            0
review_id                        0
review_score                     0
customer_id                      0
order_status                     0
product_category_name         1598
product_name_lenght           1598
product_description_lenght    1598
product_weight_g                18
product_length_cm               18
product_height_cm               18
product_width_cm                18
dtype: int64

In [105]:
df_concat.dropna(how='any',axis=0,inplace=True)

In [106]:
df_concat.drop_duplicates(subset ="order_id",keep = False, inplace = True)

In [108]:
df = df_concat[['product_id','customer_id','review_score']]

In [109]:
df

,product_id,customer_id,review_score
0,4244733e06e7ecb4970a6e2683c13e61,3ce436f183e68e07877b285a838db11a,5
1,4244733e06e7ecb4970a6e2683c13e61,e6eecc5a77de221464d1c4eaff0a9b64,5
2,4244733e06e7ecb4970a6e2683c13e61,4ef55bf80f711b372afebcb7c715344a,4
3,4244733e06e7ecb4970a6e2683c13e61,30407a72ad8b3f4df4d15369126b20c9,5
4,4244733e06e7ecb4970a6e2683c13e61,91a792fef70ecd8cc69d3c7feb3d12da,5
...,...,...,...
112367,4cc4d02efc8f249c13355147fb44e34d,050309b91cc5e04e68841938e7984aaf,5
112368,b10ecf8e33aaaea419a9fa860ea80fb5,11e0f43ab4e2d2c48348dd9332c0ef80,4
112369,dd469c03ad67e201bc2179ef077dcd48,dec8952e97ef6124259c56914fb3569c,5
112370,bbe7651fef80287a816ead73f065fc4b,a5201e1a6d71a8d21e869151bd5b4085,4


# Filtros Colaborativoss

# Itens Mais Popualres -  Somatório de Avaliações


In [125]:
ratings_explicit = df[df['review_score'] != 0]
ratings_sum = pd.DataFrame(ratings_explicit.groupby(['product_id'])['review_score'].sum())
top10 = ratings_sum.sort_values('review_score', ascending = False).head(10)
top10.rename(columns={'review_score': 'review_score_sum'},inplace=True)
top10

,review_score_sum
product_id,
99a4788cb24856965c36a24e339b6058,1547
aca2eb7d00ea1a7b8ebd4e68314663af,1449
d1c427060a0f73f6b889a5c7c61f2ac4,1248
53b36df67ebb7c41585e8d54d6772e08,1237
422879e10f46682990de24d770e7f83d,1065
154e7e31ebfa092203795c972e5804a6,1053
3dd2a17168ec895c781a9191c1e95ad7,1036
2b4609f8948be18874494203496bc318,994
389d119b48cf3043d311335e499d9c6b,954


# Itens Mais Popualres -  Melhor Média de Avaliação



In [126]:
ratings_explicit = df[df['review_score'] != 0]
ratings_sum = pd.DataFrame(ratings_explicit.groupby(['product_id'])['review_score'].mean())
top10 = ratings_sum.sort_values('review_score', ascending = False).head(10)
top10.rename(columns={'review_score': 'review_score_mean'},inplace=True)
top10

,review_score_mean
product_id,
00066f42aeeb9f3007548bb9d3f33c38,5.0
652c030867f364b558eb9f7dcbcf608d,5.0
cae58f36738671651f3d19fee286f556,5.0
cae2e38942c8489d9d7a87a3f525c06b,5.0
64f1126c9715d5394b7301934c6833f0,5.0
64fbadb8e3f6a0ac76c38ab230d661f9,5.0
cab49aa7c76189e7e6d55ad8c7f9eb91,5.0
6510b9320992123556a40f98806e512a,5.0
65194d9ad03e8206e3a9848f405942f1,5.0


# Itens Mais Populares - Maior Quantidade de Usuários que Avaliaram

In [121]:
count = df.groupby(by='product_id', as_index=False) \
  .agg({'customer_id': pd.Series.nunique}) \
  .rename(columns={'customer_id': 'customer_id_count'})
buy_count = count.set_index('product_id')
top10 = buy_count.sort_values('customer_id_count', ascending = False).head(10)
top10

,customer_id_count
product_id,
99a4788cb24856965c36a24e339b6058,388
aca2eb7d00ea1a7b8ebd4e68314663af,345
d1c427060a0f73f6b889a5c7c61f2ac4,288
53b36df67ebb7c41585e8d54d6772e08,287
422879e10f46682990de24d770e7f83d,248
2b4609f8948be18874494203496bc318,246
154e7e31ebfa092203795c972e5804a6,243
3dd2a17168ec895c781a9191c1e95ad7,238
389d119b48cf3043d311335e499d9c6b,225


# Matriz de Recomendação

In [15]:
def start_pipeline(dataf):
    return dataf.copy()

def get_product_counts(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'customer_id': pd.Series.nunique}) \
    .rename(columns={'customer_id': 'customer_id_count'}) \
    .set_index('product_id') \
    .sort_values('customer_id_count', ascending = False)

def get_avg_ratings(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'review_score': np.mean}) \
    .rename(columns={'review_score': 'review_score_avg'}) \
    .set_index('product_id') \
    .sort_values('review_score_avg', ascending = False)

def get_n_top_values(dataf, n):
  return dataf.head(n) \
    .index \
    .tolist()

top_product_ratings = df \
  .pipe(start_pipeline) \
  .pipe(get_avg_ratings) \
  .pipe(get_n_top_values, n=1500)

top_products_sellers = df \
  .pipe(start_pipeline) \
  .pipe(get_product_counts) \
  .pipe(get_n_top_values, n=10)

top_product_ratings[:10], top_products_sellers

(['00066f42aeeb9f3007548bb9d3f33c38',
  '60314cbf527af04ebc935be7c02dc1cd',
  '5fe23e6307c231e287f13c0a4be06c88',
  '5fe490e61e7a37f2f0d1ad1771ac027d',
  'c696cd2db7870be08c37822f7c0ef831',
  '5fef486f2057e10fd91e167348812b7a',
  '5ff4076c0f01eeba4f728c9e3fa2653c',
  '5ff59c5f2db7600fa8143442c8b1e4f6',
  'c68b419d9c6038271b85bac98adb0fc9',
  '5ffa5bcfb017e836862c86ec3fbf9045'],
 ['99a4788cb24856965c36a24e339b6058',
  'aca2eb7d00ea1a7b8ebd4e68314663af',
  '422879e10f46682990de24d770e7f83d',
  'd1c427060a0f73f6b889a5c7c61f2ac4',
  '389d119b48cf3043d311335e499d9c6b',
  '53b36df67ebb7c41585e8d54d6772e08',
  '368c6c730842d78016ad823897a372db',
  '53759a2ecddad2bb87a079a1f1519f73',
  '154e7e31ebfa092203795c972e5804a6',
  '2b4609f8948be18874494203496bc318'])

In [16]:
df \
  .pipe(start_pipeline) \
  .pipe(get_avg_ratings)

,review_score_avg
product_id,
00066f42aeeb9f3007548bb9d3f33c38,5.0
60314cbf527af04ebc935be7c02dc1cd,5.0
5fe23e6307c231e287f13c0a4be06c88,5.0
5fe490e61e7a37f2f0d1ad1771ac027d,5.0
c696cd2db7870be08c37822f7c0ef831,5.0
...,...
336b5407e252cac6f4e4270979cec8ad,1.0
86304b9e4529f2686bc91f37c6392801,1.0
e09cef1f7f2bc3a9c9d996608944de19,1.0


In [17]:
matriz_de_relacionamento_top_product_ratings = df[df["product_id"].isin(top_product_ratings)] \
  .pivot_table(index="product_id", columns="customer_id", values="review_score").fillna(0)

matriz_de_relacionamento_top_product_ratings.shape

(1500, 1934)

In [18]:
cos_products = cosine_similarity(matriz_de_relacionamento_top_product_ratings)
print(cos_products)
cos_products.shape

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


(1500, 1500)

In [19]:
def recomendar(id_produto):
  print(f"Produto de interesse: {id_produto}")
  try:
    produto_idx = matriz_de_relacionamento_top_product_ratings.index.tolist().index(id_produto)
    closest_10_products = np.argsort(-cos_products[produto_idx])[1:11]
    print(f"Retornando 10 Produtos mais próximos de: {id_produto}.")
    return matriz_de_relacionamento_top_product_ratings.index[closest_10_products].tolist()
  except ValueError as e:
    print(f"{id_produto}, não está incluso na matriz de recomendação. Retornando os 10 mais avaliados")
    return top_products_sellers

In [20]:
id_livro_interesse = "00066f42aeeb9f3007548bb9d3f33c38"
ids_10_mais_proximos = recomendar(id_livro_interesse)
ids_10_mais_proximos

Produto de interesse: 00066f42aeeb9f3007548bb9d3f33c38
Retornando 10 Produtos mais próximos de: 00066f42aeeb9f3007548bb9d3f33c38.


['be70253973dbb353054c814633a5d4e4',
 'be6ae1b41a4ba652f1947e0fae960d44',
 'be68ca1484d5bcc92f0949413e5eb445',
 'be59f0b9bd1d08d096188f98168fc987',
 'be5679ce5a2343c40455223d7bef6dfb',
 'be55ae76be8824525f40695d49a5a7b5',
 'be4fa7795030e85bb9fe8285405515fa',
 'be4e205f4a7283ce05c3b9365d283adc',
 'be4daa163f6e100a73d7cddcf2a4c4de',
 'be4cf22bb84e3fd215db5076e3047d51']

In [21]:
id_produto_novo = "id_novo"
ids_10_mais_vendidos = recomendar(id_produto_novo)
ids_10_mais_vendidos

Produto de interesse: id_novo
id_novo, não está incluso na matriz de recomendação. Retornando os 10 mais avaliados


['99a4788cb24856965c36a24e339b6058',
 'aca2eb7d00ea1a7b8ebd4e68314663af',
 '422879e10f46682990de24d770e7f83d',
 'd1c427060a0f73f6b889a5c7c61f2ac4',
 '389d119b48cf3043d311335e499d9c6b',
 '53b36df67ebb7c41585e8d54d6772e08',
 '368c6c730842d78016ad823897a372db',
 '53759a2ecddad2bb87a079a1f1519f73',
 '154e7e31ebfa092203795c972e5804a6',
 '2b4609f8948be18874494203496bc318']

In [22]:
def recomendar(id_produto):
  print(f"Produto de interesse: {id_produto}")
  if id_produto in matriz_de_relacionamento_top_product_ratings.index.tolist():
    livro_idx = matriz_de_relacionamento_top_product_ratings.index.tolist().index(id_produto)
    closest_10_products = np.argsort(-cos_products[livro_idx])[1:11]
    print(f"Retornando 10 produtos mais próximos de: {id_produto}.")
    return matriz_de_relacionamento_top_product_ratings.index[closest_10_products].tolist()
  else:
    print(f"{id_produto}, não está incluso na matriz de recomendação. Retornando os 10 mais avaliados")
    return top_products_sellers

In [23]:
id_livro_interesse = "00066f42aeeb9f3007548bb9d3f33c38"
ids_10_mais_proximos = recomendar(id_livro_interesse)
ids_10_mais_proximos

Produto de interesse: 00066f42aeeb9f3007548bb9d3f33c38
Retornando 10 produtos mais próximos de: 00066f42aeeb9f3007548bb9d3f33c38.


['be70253973dbb353054c814633a5d4e4',
 'be6ae1b41a4ba652f1947e0fae960d44',
 'be68ca1484d5bcc92f0949413e5eb445',
 'be59f0b9bd1d08d096188f98168fc987',
 'be5679ce5a2343c40455223d7bef6dfb',
 'be55ae76be8824525f40695d49a5a7b5',
 'be4fa7795030e85bb9fe8285405515fa',
 'be4e205f4a7283ce05c3b9365d283adc',
 'be4daa163f6e100a73d7cddcf2a4c4de',
 'be4cf22bb84e3fd215db5076e3047d51']

In [24]:
id_livro_novo = "id_novo"
ids_10_mais_vendidos = recomendar(id_livro_novo)

ids_10_mais_vendidos

Produto de interesse: id_novo
id_novo, não está incluso na matriz de recomendação. Retornando os 10 mais avaliados


['99a4788cb24856965c36a24e339b6058',
 'aca2eb7d00ea1a7b8ebd4e68314663af',
 '422879e10f46682990de24d770e7f83d',
 'd1c427060a0f73f6b889a5c7c61f2ac4',
 '389d119b48cf3043d311335e499d9c6b',
 '53b36df67ebb7c41585e8d54d6772e08',
 '368c6c730842d78016ad823897a372db',
 '53759a2ecddad2bb87a079a1f1519f73',
 '154e7e31ebfa092203795c972e5804a6',
 '2b4609f8948be18874494203496bc318']